This notebook checks whether PXD projects are reprocessed by ionbot and stored in the vm

In [2]:
import re
from pathlib import Path
import glob
import pandas as pd

In [2]:
found_pxd = ['PXD004051', 'PXD006421', 'PXD005620', 'PXD002676', 'PXD001952', 'PXD007885', 'PXD007906', 'PXD021281', 'PXD006653', 'PXD002842', 'PXD003594', 'PXD006035', 'PXD008719', 'PXD001668', 'PXD009185', 'PXD005507', 'PXD001247', 'PXD007837', 'PXD008035', 'PXD001974', 'PXD002057', 'PXD008381', 'PXD010012', 'PXD005045', 'PXD009933', 'PXD002032', 'PXD010538', 'PXD006614', 'PXD009600', 
'PXD009442', 'PXD010306', 'PXD009557', 'PXD009560', 'PXD003252', 'PXD003587', 'PXD007543', 'PXD008693', 'PXD003362', 'PXD011580', 'PXD010256', 'PXD000426', 'PXD009861', 'PXD006419', 'PXD008967', 'PXD000612', 'PXD001592', 'PXD003438', 'PXD004824', 'PXD014300', 'PXD018450', 'PXD018625', 'PXD017391', 'PXD018918', 'PXD011961', 'PXD013541', 'PXD018066', 'PXD016742', 'PXD014777', 'PXD017898', 'PXD014381', 'PXD016119', 'PXD015571', 'PXD009149']
found_pxd = pd.Series(found_pxd)
#found_pxd.to_csv("filter_found_pxd2.csv", index = False)

In [3]:
data = pd.read_csv("/home/compomics/Sam/git/python/master_thesis/Metadata/cellosaurus_webscraping_filtered_update.csv", sep = ";", usecols = ["PXD", "Cell", "Useable"], encoding = 'latin1')
data2 = pd.read_csv("filter_found_pxd.csv", sep = ";", usecols = ["PXD", "Cell", "Useable"])
new_pxd = list(data[data.Useable == "yes"].PXD)
new_pxd2 = list(data2[data2.Useable == "yes"].PXD)
#print(len(new_pxd), len(new_pxd2))
total_projects = new_pxd + new_pxd2
len(total_projects)

108

In [3]:
root = Path("/home/compomics/mounts")

pxd_dir = glob.glob("/home/compomics/mounts/*/*/PRIDE_DATA/*")
print(pxd_dir[:5])
len(pxd_dir)

['/home/compomics/mounts/conode53/pride2/PRIDE_DATA/PXD002823', '/home/compomics/mounts/conode53/pride2/PRIDE_DATA/PXD001857', '/home/compomics/mounts/conode53/pride2/PRIDE_DATA/PXD000272', '/home/compomics/mounts/conode53/pride2/PRIDE_DATA/PXD004019', '/home/compomics/mounts/conode53/pride2/PRIDE_DATA/PXD004113']


2475

In [4]:
available_pxd = [x.split('/')[-1] for x in pxd_dir]
len(available_pxd)

2475

In [5]:
#check whether previously unprocessed projects, are processed
unprocessed = ['PXD000535',
'PXD001441',
'PXD001468',
'PXD001511',
'PXD002117',
'PXD002613',
'PXD003530',
'PXD003596',
'PXD003668',
'PXD003790',
'PXD003896',
'PXD003903',
'PXD004182',
'PXD004452',
'PXD004900',
'PXD004940',
'PXD005354',
'PXD005572',
'PXD005912',
'PXD005946',
'PXD005940',
'PXD005942',
'PXD006112',
'PXD009686',
'PXD013615',
'PXD008222',
'PXD000612',
'PXD001668',
'PXD001952',
'PXD001974',
'PXD002676',
'PXD004051',
'PXD005507',
'PXD006653',
'PXD009185']

new = [x for x in unprocessed if x in available_pxd]

print(f"{len(new)}/{len(unprocessed)} projects reprocessed.")

35/35 projects reprocessed.


In [6]:
PXD_reprocessed = []
dir_PXD_reprocessed = []
regex = re.compile(r"(PXD)(\d+)")
for i in pxd_dir:
    result = regex.findall(i)
    if result != []:    
        #PXD_reprocessed.append("".join(result[0]))
        if "".join(result[0]) in unprocessed:
            dir_PXD_reprocessed.append(i)
            PXD_reprocessed.append("".join(result[0]))



In [7]:
print(len(PXD_reprocessed), PXD_reprocessed[:5])
print(len(dir_PXD_reprocessed), dir_PXD_reprocessed[:5])

35 ['PXD001468', 'PXD004051', 'PXD000535', 'PXD002676', 'PXD001952']
35 ['/home/compomics/mounts/conode53/pride/PRIDE_DATA/PXD001468', '/home/compomics/mounts/conode53/pride/PRIDE_DATA/PXD004051', '/home/compomics/mounts/conode53/pride/PRIDE_DATA/PXD000535', '/home/compomics/mounts/conode53/pride/PRIDE_DATA/PXD002676', '/home/compomics/mounts/conode53/pride/PRIDE_DATA/PXD001952']


Make excell file for manual file annotation (*file_annotation_update.csv*)

In [9]:
import openpyxl, os
check1 = {"PXD": 0, "RAW": 0}
wb = openpyxl.Workbook()
sheet = wb["Sheet"]
sheet["A1"], sheet["B1"], sheet["C1"], sheet["D1"], sheet["E1"] = "PXD", "RAW", "cell_line", "disease", "tissue_type"

for i in dir_PXD_reprocessed:
    check1['PXD'] += 1
    print(i)
    sheet.cell(row = sheet.max_row + 2, column = 1).value = os.path.basename(i)
    for count, x in enumerate(glob.glob(f"{i}/RAW/*")):
        check1['RAW'] += 1
        if count == 0:
            sheet.cell(row = sheet.max_row, column = 2). value = os.path.basename(x)
            sheet.cell(row = sheet.max_row, column = 1).value = os.path.basename(i)
            continue
        sheet.cell(row = sheet.max_row + 1, column = 2). value = os.path.basename(x)
        sheet.cell(row = sheet.max_row, column = 1).value = os.path.basename(i)

wb.save("annotation_excel4.xlsx")

print(check1)

/home/compomics/mounts/conode53/pride/PRIDE_DATA/PXD001468
/home/compomics/mounts/conode53/pride/PRIDE_DATA/PXD004051
/home/compomics/mounts/conode53/pride/PRIDE_DATA/PXD000535
/home/compomics/mounts/conode53/pride/PRIDE_DATA/PXD002676
/home/compomics/mounts/conode53/pride/PRIDE_DATA/PXD001952
/home/compomics/mounts/conode53/pride/PRIDE_DATA/PXD000612
/home/compomics/mounts/conode53/pride/PRIDE_DATA/PXD002117
/home/compomics/mounts/conode53/pride/PRIDE_DATA/PXD006653
/home/compomics/mounts/conode53/pride/PRIDE_DATA/PXD001511
/home/compomics/mounts/conode53/pride/PRIDE_DATA/PXD001668
/home/compomics/mounts/conode53/pride/PRIDE_DATA/PXD009185
/home/compomics/mounts/conode53/pride/PRIDE_DATA/PXD005507
/home/compomics/mounts/conode53/pride/PRIDE_DATA/PXD001974
/home/compomics/mounts/conode55/pride/PRIDE_DATA/PXD005572
/home/compomics/mounts/conode55/pride/PRIDE_DATA/PXD003903
/home/compomics/mounts/conode55/pride/PRIDE_DATA/PXD003896
/home/compomics/mounts/conode55/pride/PRIDE_DATA/PXD0026